In [1]:
import cadquery as cq
from jupyter_cadquery.cadquery import Assembly, Part, Edges, Faces, show
from jupyter_cadquery import auto_show

base_width = 38
base_length = 64
gap = 6
holder_length = 22

xy = cq.Workplane("YX")

Removing auto display for cadquery Workplane and Shape


In [2]:
import sidecar
sc = sidecar.Sidecar()

In [5]:
fullbase = (xy
    .vLineTo(-base_length/2 + gap/2, forConstruction=True)
    .hLineTo(-gap/2)
    .vLineTo(-base_length/2)
    .hLineTo(-base_width/2)
    .vLineTo(-base_length/2 + 10)
    .hLineTo(-base_width/2 + gap/2)
    .vLineTo(holder_length / 2 - 2)
    .hLineTo(-base_width/2)
    .vLineTo(holder_length / 2)
    .threePointArc((-base_width*3/8, base_length*3/8), (-gap/2, base_length/2))
    .vLineTo((base_length-gap)/2)
    .hLineTo(0)
    .mirrorY()
)
Edges(fullbase)

In [6]:
show(_)

In [8]:
from jupyter_cadquery.cadquery.cad_objects import _is_wire_list, _is_edge_list, _edge_list_to_assembly, _wire_list_to_assembly,_to_occ
from cadquery import Edge
#a = _edge_list_to_assembly(fullbase.edges())

In [28]:
show(_wire_list_to_assembly(fullbase))

In [27]:
show(_edge_list_to_assembly(box.edges()))

In [ ]:
_.

In [16]:
_is_wire_list(fullbase)

True

In [43]:
fullbase.objects

In [32]:
Edge?

Init signature: Edge(obj)
Docstring:      A trimmed curve that represents the border of a face
File:           /opt/miniconda/envs/pycq/lib/python3.6/site-packages/cadquery/occ_impl/shapes.py
Type:           type
Subclasses:     


In [44]:
all_edges = []
for edges in fullbase.objects:
    all_edges += edges.Edges()

In [49]:
fullbase.objects

True

In [6]:
b = cq.Workplane('XY')
box = b.box(10, 20, 30).edges(">X or <X").fillet(2)
box.edges().objects

In [7]:
Edges(box.edges()).shape

> /Users/bernhard/Development/cadquery/jupyter-cadquery/jupyter_cadquery/cadquery/cad_objects.py(71)_to_occ()
-> if _is_wire_list(cad_obj):


(Pdb)  c


[class<'TopoDS_Edge'; id:661321329>,
 class<'TopoDS_Edge'; id:661324769>,
 class<'TopoDS_Edge'; id:661326177>,
 class<'TopoDS_Edge'; id:661324097>,
 class<'TopoDS_Edge'; id:661327585>,
 class<'TopoDS_Edge'; id:661321681>,
 class<'TopoDS_Edge'; id:661327937>,
 class<'TopoDS_Edge'; id:661325121>,
 class<'TopoDS_Edge'; id:661328993>,
 class<'TopoDS_Edge'; id:661326529>,
 class<'TopoDS_Edge'; id:661329345>,
 class<'TopoDS_Edge'; id:661324449>,
 class<'TopoDS_Edge'; id:1122088355>,
 class<'TopoDS_Edge'; id:661320913>,
 class<'TopoDS_Edge'; id:1122061955>,
 class<'TopoDS_Edge'; id:1122101907>,
 class<'TopoDS_Edge'; id:661322385>,
 class<'TopoDS_Edge'; id:1122102931>,
 class<'TopoDS_Edge'; id:661319473>,
 class<'TopoDS_Edge'; id:661325825>,
 class<'TopoDS_Edge'; id:661326881>,
 class<'TopoDS_Edge'; id:661320545>,
 class<'TopoDS_Edge'; id:661327233>,
 class<'TopoDS_Edge'; id:661322033>,
 class<'TopoDS_Edge'; id:661328289>,
 class<'TopoDS_Edge'; id:661322737>,
 class<'TopoDS_Edge'; id:661328641

In [9]:
fullbase

# Analysis (done)

In [ ]:
from OCC.Core.BRepBuilderAPI import (BRepBuilderAPI_MakeVertex, BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire)
from OCC.Extend.TopologyUtils import WireExplorer, TopologyExplorer, dump_topology_to_string                                 
import OCC.Core.TopAbs as ta 
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods_Vertex, topods_Edge
from OCC.Core.BRep import BRep_Tool
from cadquery import Wire
from cadquery.occ_impl.shapes import downcast, inverse_shape_LUT


def _entities(self, topo_type):

    out = {}  # using dict to prevent duplicates

    explorer = TopExp_Explorer(self, inverse_shape_LUT[topo_type])

    while explorer.More():
        item = explorer.Current()
        out[item.__hash__()] = item
        explorer.Next()

    return list(out.values())

def reverseEdge(edge):
    brt = BRep_Tool()
    vertices = _entities(edge, "Vertex")
    pnts = [brt.Pnt(topods_Vertex(v)) for v in vertices]

    return BRepBuilderAPI_MakeEdge(pnts[1], pnts[0]).Edge()

def reverseWire(wire):
    wire_builder = BRepBuilderAPI_MakeWire()
    we = WireExplorer(wire.wrapped)
    edges = reversed(list(we.ordered_edges()))
    for edge in edges:
        wire_builder.Add(reverseEdge(edge))
    result = wire_builder.Wire()
    return wire.__class__(result)

def mirrorY(wire):
    # convert edges to a wire, if there are pending edges
    n = wire.wire(forConstruction=False)

    # attempt to consolidate wires together.
    consolidated = n.consolidateWires()

    mirroredWires = wire.plane.mirrorInPlane(consolidated.wires().vals(), 'Y')
    for w in mirroredWires:
        reversed_wire = reverseWire(w)
        consolidated.objects.append(reversed_wire)
        consolidated._addPendingWire(reversed_wire)

    # attempt again to consolidate all of the wires
    return consolidated.consolidateWires()

def debug_edge(edge):
    for vertex in edge.Vertices():
        print('({:>6} {:>6} {:>6})'.format(vertex.X, vertex.Y, vertex.Z), end=" ")
    print()
    
def debug_wire(wire):
    for edge in wire.Edges():
        print(edge.wrapped.Orientation(), end= " ")
        debug_edge(edge)
        
fullbase = mirrorY(xy
    .vLineTo(-base_length/2 + gap/2, forConstruction=True)
    .hLineTo(-gap/2)
    .vLineTo(-base_length/2)
    .hLineTo(-base_width/2)
    .vLineTo(-base_length/2 + 10)
    .hLineTo(-base_width/2 + gap/2)
    .vLineTo(holder_length / 2 - 2)
    .hLineTo(-base_width/2)
    .vLineTo(holder_length / 2)
    .threePointArc((-base_width*3/8, base_length*3/8), (-gap/2, base_length/2))
    .vLineTo((base_length-gap)/2)
    .hLineTo(0)
)
# show(Wires(my_mirrorY(base)), sidecar=sc)
show(fullbase, sidecar=sc)

In [ ]:
for x in fullbase.objects:
    debug_wire(x)
    print("---")

In [ ]:
from OCC.Extend.TopologyUtils import TopologyExplorer, is_edge, is_wire, discretize_edge, discretize_wire

In [ ]:
discretize_edge(base)

In [ ]:
base = (xy
    .vLineTo(-base_length/2 + gap/2, forConstruction=True)
    .hLineTo(-gap/2)
    .vLineTo(-base_length/2)
    .hLineTo(-base_width/2)
    .vLineTo(-base_length/2 + 10)
    .hLineTo(-base_width/2 + gap/2)
    .vLineTo(holder_length / 2 - 2)
    .hLineTo(-base_width/2)
    .vLineTo(holder_length / 2)
    .threePointArc((-base_width*3/8, base_length*3/8), (-gap/2, base_length/2))
    .vLineTo((base_length-gap)/2)
    .hLineTo(0)
)

n = base.wire(forConstruction=False)
debug_wire(n.objects[0])
n

In [ ]:
consolidated = n.consolidateWires()
consolidated

In [ ]:
mirroredWires = base.plane.mirrorInPlane(consolidated.wires().vals(), 'Y')
debug_wire(mirroredWires[0])

In [ ]:
mirroredWires[0].wrapped.Reverse()

In [ ]:
consolidated.objects.append(mirroredWires[0])
debug_wire(consolidated.consolidateWires().objects[0])

In [ ]:
from OCC.Extend.TopologyUtils import WireExplorer, TopologyExplorer, dump_topology_to_string

In [ ]:
te = TopologyExplorer(n.objects[0].wrapped)
list(te.ordered_vertices_from_wire(n.objects[0].wrapped))

In [ ]:
list(te.ordered_vertices_from_wire(n.objects[0].wrapped.Reversed()))

In [ ]:
x = mirroredWires[0].wrapped

In [ ]:
dump_topology_to_string(x)

In [ ]:
x.Reverse()
dump_topology_to_string(x)

In [ ]:
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeWire

def reverse(cls, listOfWires):
    wire_builder = BRepBuilderAPI_MakeWire()
    for wire in listOfWires:
        print(wire.wrapped)
        print(wire.wrapped.Reversed())
        wire.wrapped.Reverse()
        print(wire)
        wire_builder.Add(wire.wrapped.Reversed())

    return cls(wire_builder.Wire())

rev = reverse(cq.Wire, mirroredWires)
debug_wire(rev)

In [ ]:
def mirrorY(workplane):
    n = workplane.wire(forConstruction=False)

    # attempt to consolidate wires together.
    consolidated = n.consolidateWires()

    mirroredWires = workplane.plane.mirrorInPlane(consolidated.wires().vals(), 'Y')

    for w in mirroredWires:
        consolidated.objects.append(w)
        consolidated._addPendingWire(w)

    # attempt again to consolidate all of the wires
    return consolidated.consolidateWires()

In [ ]:
base = (xy
    .vLineTo(-base_length/2 + gap/2, forConstruction=True)
    .hLineTo(-gap/2)
    .vLineTo(-base_length/2)
    .hLineTo(-base_width/2)
    .vLineTo(-base_length/2 + 10)
    .hLineTo(-base_width/2 + gap/2)
    .vLineTo(holder_length / 2 - 2)
    .hLineTo(-base_width/2)
    .vLineTo(holder_length / 2)
    .threePointArc((-base_width*3/8, base_length*3/8), (-gap/2, base_length/2))
    .vLineTo((base_length-gap)/2)
    .hLineTo(0)
    .mirrorY()
)

# Wires(base.mirrorY())
len(base.edges().objects)

In [ ]:
def mirrorY(workplane):
    n = workplane.wire(forConstruction=False)
    print(n)

    # attempt to consolidate wires together.
    consolidated = n.consolidateWires()

    mirroredWires = workplane.plane.mirrorInPlane(consolidated.wires().vals(), 'Y')
    print(mirroredWires)
    for w in mirroredWires:
        consolidated.objects.append(w)
        consolidated._addPendingWire(w)

    # attempt again to consolidate all of the wires
    return consolidated.consolidateWires()

In [ ]:
base = (xy
    .vLineTo(-base_length/2 + gap/2, forConstruction=True)
    .hLineTo(-gap/2)
    .vLineTo(-base_length/2)
    .hLineTo(-base_width/2)
    .vLineTo(-base_length/2 + 10)
    .hLineTo(-base_width/2 + gap/2)
    .vLineTo(holder_length / 2 - 2)
    .hLineTo(-base_width/2)
    .vLineTo(holder_length / 2)
    .threePointArc((-base_width*3/8, base_length*3/8), (-gap/2, base_length/2))
    .vLineTo((base_length-gap)/2)
    .hLineTo(0)
)
# n = mirrorY(base).objects
print(base.wire(forConstruction=False).consolidateWires().wires().vals())

In [ ]:
[(v.X, v.Y, v.Z) for v in o.Vertices()]

In [ ]:
[(v.X, v.Y, v.Z) for v in m.Vertices()]

In [ ]:
[(v.X, v.Y, v.Z) for v in n[0].Vertices()]

In [ ]:
print(base.wire(forConstruction=False).consolidateWires().wires().vals())

In [ ]:
base = (xy
    .vLineTo(-base_length/2 + gap/2, forConstruction=True)
    .hLineTo(-gap/2)
    .vLineTo(-base_length/2)
    .hLineTo(-base_width/2)
    .vLineTo(-base_length/2 + 10)
    .hLineTo(-base_width/2 + gap/2)
    .vLineTo(holder_length / 2 - 2)
    .hLineTo(-base_width/2)
    .vLineTo(holder_length / 2)
    .threePointArc((-base_width*3/8, base_length*3/8), (-gap/2, base_length/2))
    .vLineTo((base_length-gap)/2)
    .hLineTo(0)
    .wire()
)


# Analysis

In [ ]:
import cadquery as cq

base_width = 38
base_length = 64
gap = 6
holder_length = 22

xy = cq.Workplane("YX")

def my_mirrorY(workplane):
    n = workplane.wire(forConstruction=False)
    mirroredWires = workplane.plane.mirrorInPlane(n.wires().vals(), 'Y')
    for w in mirroredWires:
        n.objects.append(w)
        n._addPendingWire(w)
    return n

In [ ]:
base1 = (xy
    .vLineTo(-base_length/2 + gap/2, forConstruction=True)
    .hLineTo(-gap/2)
    .vLineTo(-base_length/2)
    .hLineTo(-base_width/2)
    .vLineTo(-base_length/2 + 10)
    .hLineTo(-base_width/2 + gap/2)
    .vLineTo(holder_length / 2 - 2)
    .hLineTo(-base_width/2)
    .vLineTo(holder_length / 2)
    .threePointArc((-base_width*3/8, base_length*3/8), (-gap/2, base_length/2))
    .vLineTo((base_length-gap)/2)
    .hLineTo(0)
    .mirrorY()
)

In [ ]:
base2 = my_mirrorY(xy
    .vLineTo(-base_length/2 + gap/2, forConstruction=True)
    .hLineTo(-gap/2)
    .vLineTo(-base_length/2)
    .hLineTo(-base_width/2)
    .vLineTo(-base_length/2 + 10)
    .hLineTo(-base_width/2 + gap/2)
    .vLineTo(holder_length / 2 - 2)
    .hLineTo(-base_width/2)
    .vLineTo(holder_length / 2)
    .threePointArc((-base_width*3/8, base_length*3/8), (-gap/2, base_length/2))
    .vLineTo((base_length-gap)/2)
    .hLineTo(0)
)


In [ ]:
Wires(base2)

In [ ]:
v1 = [[(v.X, v.Y, v.Z) for v in edge.Vertices()] for edge in base1.edges().objects]
v2 = [[(v.X, v.Y, v.Z) for v in edge.Vertices()] for edge in base2.edges().objects]
print(str(v1) == str(v2))
v1

In [ ]:
str(v1) == str(v2)

In [ ]:
w1 = [obj.wrapped for obj in base1.objects]
w1

In [ ]:
w2 = [obj.wrapped for obj in base2.objects]
w2

In [ ]:
from OCC.Extend.TopologyUtils import discretize_wire
dw1 = [discretize_wire(w) for w in w1]
dw2 = [discretize_wire(w) for w in w2]

In [ ]:
for i, ((x1,y1,z1),(x2,y2,y3)) in enumerate(zip(dw1[0], dw2[0]+dw2[1])):
    if i == 222:
        print("= = = = ")
    print("% 3d:" % i, x1, y1, " - ", x2, y2)

In [ ]:
dw2

In [ ]:
Wires(base1)

## Analysis 2

In [ ]:
base1 = (xy
    .vLineTo(-base_length/2 + gap/2, forConstruction=True)
    .hLineTo(-gap/2)
    .vLineTo(-base_length/2)
    .hLineTo(-base_width/2)
    .vLineTo(-base_length/2 + 10)
    .hLineTo(-base_width/2 + gap/2)
    .vLineTo(holder_length / 2 - 2)
    .hLineTo(-base_width/2)
    .vLineTo(holder_length / 2)
    .threePointArc((-base_width*3/8, base_length*3/8), (-gap/2, base_length/2))
    .vLineTo((base_length-gap)/2)
    .hLineTo(0)
)

In [ ]:
n = base1.wire(forConstruction=False)
consolidated = n.consolidateWires()

In [ ]:
n.wires().vals()

In [ ]:
for pe in n.ctx.pendingEdges:
    print(pe)
print(n.ctx.pendingWires)
print(n.objects)

In [ ]:
for pe in consolidated.ctx.pendingEdges:
    print(pe)
print(consolidated.ctx.pendingWires)
print(consolidated.objects)

In [ ]:
consolidated.wires().vals()

In [ ]:
def my_mirrorY2(workplane):
    n = workplane.wire(forConstruction=False)
    consolidated = n.consolidateWires()
    mirroredWire = workplane.plane.mirrorInPlane(consolidated.wires().vals(), 'Y')[0]
    w = Wire.assembleEdges(reversed(mirroredWire.Edges()))

    consolidated.objects.append(w)
    consolidated._addPendingWire(w)
    return consolidated.consolidateWires()

In [ ]:
base3 = (xy
    .vLineTo(-base_length/2 + gap/2, forConstruction=True)
    .hLineTo(-gap/2)
    .vLineTo(-base_length/2)
    .hLineTo(-base_width/2)
    .vLineTo(-base_length/2 + 10)
    .hLineTo(-base_width/2 + gap/2)
    .vLineTo(holder_length / 2 - 2)
    .hLineTo(-base_width/2)
    .vLineTo(holder_length / 2)
    .threePointArc((-base_width*3/8, base_length*3/8), (-gap/2, base_length/2))
    .vLineTo((base_length-gap)/2)
    .hLineTo(0)
)


In [ ]:
c = my_mirrorY2(base3)

In [ ]:
Wires(c)

In [ ]:
c[0].Edges()

In [ ]:
Wire.assembleEdges(reversed(c[0].Edges()))

In [ ]:
print(inspect.getsource(c[0]._entities))

In [ ]:
o = c.objects[0]
o.Edges()

In [ ]:
my_mirrorY2(base3)[0], base3.wire()

In [ ]:
Wires(base3.consolidateWires())

In [ ]:
base4 = base3.consolidateWires()

In [ ]:
base3.ctx.pendingWires

In [ ]:
base4.ctx.pendingWires

In [ ]:
base3.objects

In [ ]:
base4.objects

In [ ]:
from cadquery import Wire

In [ ]:
import inspect
print(inspect.getsource(Wire.combine))

In [ ]:
base3.objects[0].wrapped

In [ ]:
from OCC.Core.ShapeFix import ShapeFix_Wire

yz = cq.Workplane("YZ")
xy = cq.Workplane("YX")

diam = 5.2
base_thickness = 2
base_width = 38
base_length = 64
gap = 6

holder_thickness = 1
holder_length = 22

holder_width = diam + 2 * holder_thickness
holder_height = diam + holder_thickness

side_margin = base_width/2 - holder_width
bottom_margin = 15

base = (xy
    .vLineTo(-base_length/2 + gap/2, forConstruction=True)
    .hLineTo(-gap/2)
    .vLineTo(-base_length/2)
    .hLineTo(-base_width/2)
    .vLineTo(-base_length/2 + 10)
    .hLineTo(-base_width/2 + gap/2)
    .vLineTo(holder_length / 2 - 2)
    .hLineTo(-base_width/2)
    .vLineTo(holder_length / 2)
    .threePointArc((-base_width*3/8, base_length*3/8), (-gap/2, base_length/2))
    .vLineTo((base_length-gap)/2)
    .hLineTo(0).mirrorY()
)

w = base.val()
f = cq.Face.makeFromWires(w)

sf = ShapeFix_Wire(w.wrapped,f.wrapped,1e-6)
sf.FixReorder()
base2 = cq.Shape(sf.Wire())

In [ ]:
Wires(base2)

In [ ]:
list(enumerate(w))[0]

In [ ]:
from OCC.Extend.TopologyUtils import discretize_wire

base = (xy
    .vLineTo(-base_length/2 + gap/2, forConstruction=True)
    .hLineTo(-gap/2)
    .vLineTo(-base_length/2)
    .hLineTo(-base_width/2)
    .vLineTo(-base_length/2 + 10)
    .hLineTo(-base_width/2 + gap/2)
    .vLineTo(holder_length / 2 - 2)
    .hLineTo(-base_width/2)
    .vLineTo(holder_length / 2)
    .threePointArc((-base_width*3/8, base_length*3/8), (-gap/2, base_length/2))
    .vLineTo((base_length-gap)/2)
    .hLineTo(0).mirrorY()
)
w = discretize_wire(base.objects[0].wrapped)
for i,(x,y,z) in enumerate(w[210:234]):
    print("%03d %6.4f  %6.4f  %6.4f" % (i+210, x, y, z))

In [ ]:
w

In [ ]:
import cadquery as cq
from jupyter_cadquery.cadquery import Wires, Faces, show

yz = cq.Workplane("YZ")
xy = cq.Workplane("YX")

diam = 5.2
base_thickness = 2
base_width = 38
base_length = 64
gap = 6

holder_thickness = 1
holder_length = 22

holder_width = diam + 2 * holder_thickness
holder_height = diam + holder_thickness

side_margin = base_width/2 - holder_width
bottom_margin = 15

base = (xy
    .vLineTo(-base_length/2 + gap/2, forConstruction=True)
    .hLineTo(-gap/2)
    .vLineTo(-base_length/2)
    .hLineTo(-base_width/2)
    .vLineTo(-base_length/2 + 10)
    .hLineTo(-base_width/2 + gap/2)
    .vLineTo(holder_length / 2 - 2)
    .hLineTo(-base_width/2)
    .vLineTo(holder_length / 2)
    .threePointArc((-base_width*3/8, base_length*3/8), (-gap/2, base_length/2))
    .vLineTo((base_length-gap)/2)
    .hLineTo(0).mirrorY()
)

w = base.val()
f = cq.Face.makeFromWires(w)
discretize_me = f.Wires()[0]

In [ ]:
Wires(discretize_me)

In [ ]:
w = cq.Workplane("XY")

In [ ]:
box

In [ ]:
from jupyter_cadquery.occ import show as occ_show

In [ ]:
occ_show(f.wrapped)

In [ ]:
import cadquery as cq

In [ ]:
from OCC.gp import gp_Pnt
from OCC.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire

from OCC.Core.BRep import BRep_Tool
from OCC.Core.TopoDS import topods_Vertex, topods_Edge

from OCC.Extend.TopologyUtils import WireExplorer, TopologyExplorer, dump_topology_to_string
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
r = JupyterRenderer()

In [ ]:
x = []
y = []

x.append(gp_Pnt( -29.0,  0.0, 0.0)); y.append(gp_Pnt( -29.0,  3.0, 0.0))
x.append(gp_Pnt( -29.0,  3.0, 0.0)); y.append(gp_Pnt( -32.0,  3.0, 0.0))
x.append(gp_Pnt( -32.0,  3.0, 0.0)); y.append(gp_Pnt( -32.0, 19.0, 0.0))
x.append(gp_Pnt( -32.0, 19.0, 0.0)); y.append(gp_Pnt( -22.0, 19.0, 0.0))
x.append(gp_Pnt( -22.0, 19.0, 0.0)); y.append(gp_Pnt( -22.0, 16.0, 0.0))
x.append(gp_Pnt( -22.0, 16.0, 0.0)); y.append(gp_Pnt(   9.0, 16.0, 0.0))
x.append(gp_Pnt(   9.0, 16.0, 0.0)); y.append(gp_Pnt(   9.0, 19.0, 0.0))
x.append(gp_Pnt(   9.0, 19.0, 0.0)); y.append(gp_Pnt(  11.0, 19.0, 0.0))
x.append(gp_Pnt(  11.0, 19.0, 0.0)); y.append(gp_Pnt(  32.0,  3.0, 0.0))
x.append(gp_Pnt(  32.0,  3.0, 0.0)); y.append(gp_Pnt(  29.0,  3.0, 0.0))
x.append(gp_Pnt(  29.0,  3.0, 0.0)); y.append(gp_Pnt(  29.0,  0.0, 0.0))

edges = []; 
for a,b in zip(x,y):
    edges.append(BRepBuilderAPI_MakeEdge(a,b))

edges

builder = BRepBuilderAPI_MakeWire()
for edge in edges:
    builder.Add(edge.Edge())

wire = builder.Wire()
r.DisplayShape(wire)
display(r)

In [ ]:
brt = BRep_Tool()

def reverse(wire):
    builder = BRepBuilderAPI_MakeWire()
    we = WireExplorer(wire)
    edges = [edge for edge in reversed(list(we.ordered_edges()))]
    for edge in we.ordered_edges():
        builder.Add(edge.Reversed())
    return builder.Wire()


In [ ]:
we = WireExplorer(wire)
for e in we.ordered_edges():


In [ ]:
e = list(we.ordered_edges())[0]
dump_topology_to_string(e)

dump_topology_to_string(e.Reversed())


In [ ]:
we = WireExplorer(wire)
for v in list(reversed(list(we.ordered_vertices()))):
    pnt = brt.Pnt(topods_Vertex(v))
    print(pnt.X(), -pnt.Y(), pnt.Z())

In [ ]:
dump_topology_to_string(wire)

In [ ]:
dump_topology_to_string(reverse(wire))

In [ ]:
def reverse(wire):
    wire_builder = BRepBuilderAPI_MakeWire()
    we = WireExplorer(wire.wrapped)
    edges = [edge for edge in reversed(list(we.ordered_edges()))]
    for edge in we.ordered_edges():
        print(edge.Orientation())
        print(edge.Reversed().Orientation())
        print(edge.Reversed().Vertices())
        wire_builder.Add(edge.Reversed())
    return wire.__class__(wire_builder.Wire())